In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from modelitool.simulate import Simulator

# Tutorial for sensivity analysis and parameters calibration 
This tutorial uses a practical example of linear state-space models to show how to use <code>SAnalysis</code> for sensitivity analyses, and pymoo problems for parameters optimization with **CorrAI**. 

## Load experimental data
The test study is an experimental test cell called the Armadillo Box: a demonstration building of 42 m² floor area, designed for the 2010 European Solar Decathlon by the ENSAG-GAIA-INES team. The envelope is a light wood framed construction with integrated insulation. Heating and cooling is performed by a “3 in 1” heat pump, and photovoltaic solar panels provide recharge for electric vehicles. 
Experimental data was retrieved from Simon Rouchier's github page https://github.com/srouchier/buildingenergygeeks and model.The building is monitored by a variety of sensors, but the present study only uses records of indoor temperature and prescribed heating power, in addition to weather data. 

In [ ]:
df = pd.read_csv(
    Path(r"C:/Users/thubert/PycharmProjects/corrai/tutorials/resources/statespace.csv"),
    sep=",",
    index_col=0,
    parse_dates=True, 
    decimal="."
)

In [ ]:
df["time_sec"] = df.index
inputs = df[['time_sec','T_ext', 'P_hea', 'I_sol', 'T_int']]

Time is in seconds, let's convert this index into dates using method <code>combitabconvert</code> from library modelitool.

In [ ]:
from modelitool.combitabconvert import seconds_to_datetime

In [ ]:
inputs.index = seconds_to_datetime(inputs.index, ref_year=2024)
inputs

## Create a model
We consider a simple implementation of a resistance-capacitance model for the used case. A deterministic formulation (based on Chapter 11.3  Deterministic formulation of https://buildingenergygeeks.org/a-simple-rc-model-python.html), using explicit discretisation, is proposed. Indoor and envelope temperature at time t+1 are functions of varibales at time t: 


$$
T_{i}(t+1) = T_{i}(t) + dt \cdot C_{i} \left( \frac{1}{R_{i}}(T_{e} - T_{i}) + \Phi_{h} + A_{i}\Phi_{s} \right)
$$

$$
T_{e}(t+1) = T_{e}(t) + dt \cdot C_{e} \left( \frac{1}{R_{i}}(T_{i} - T_{e}) + \frac{1}{R_{o}}(T_{o} - T_{e})+A_{e}\Phi_{s} \right)
$$

Where:
- *T_{i}(t+1)* is the internal temperature at time *t+1*,
- *T_{i}(t)* is the internal temperature at time *t*,
- *T_{e}* is the external temperature,
- *dt* is the time step,
- *Ci* and *Ce* are the heat capacitances of the interior and the envelope,
- *Re* is the resistance between the envelope and the ambient air,
- *Ri* is the resistance between the indoor temperature and the envelope,
- *Phi_{h}* is the indoor heating power,
- *Ai* and *Ae* are the solar gain coefficient of the interior and the envelope,
- *Phi_{s}* is the global horizontal solar irrandiance.

We define this model as a python class, inheriting from base class Model, with a method simulate. 
As inputs, it takes : 
- simulation options (here, let's say inputs of the models)
- a parameter dictionary containing Ri,Ro,Ci,Ce,Ai,Ae

In [ ]:
from corrai.base.model import Model
import numpy as np

class SimpleRC(Model):
    
    def simulate(self, parameter_dict, simulation_options):

        Ri, Ro, Ci, Ce, Ai, Ae = parameter_dict.values()

        df = simulation_options["dataframe"]
        time = df["time_sec"].values
        to   = df["T_ext"].values
        phi_h = df["P_hea"].values
        phi_s = df["I_sol"].values

        ti = np.zeros(len(time))
        te = np.zeros(len(time))

        ti[0] = df['T_int'][0]
        te[0] = (Ri * to[0] + Ro * ti[0]) / (Ri + Ro)

        for t in range(1, len(time)):
            dt = time[t] - time[t-1]
            ti[t] = ti[t-1] + dt / Ci * ((te[t-1] - ti[t-1]) / Ri + phi_h[t-1] + Ai * phi_s[t-1])
            te[t] = te[t-1] + dt / Ce * ((ti[t-1] - te[t-1]) / Ri + (to[t-1] - te[t-1]) / Ro + Ae * phi_s[t-1])
        
        df_out = pd.DataFrame(ti, columns=["Ti"], index=df.index)

        return df_out

We can instanciate the model, and run it with initial values as guesses.

In [ ]:
simulation_options={
    "dataframe":inputs
}

In [ ]:
parameter_dict={
    "Ri":0.01,
    "Ro":0.01,
    "Ci":1e6,
    "Ce":1e7,
    "Ai":5,
    "Ae":5
}

In [ ]:
simple_rc = SimpleRC()
result = simple_rc.simulate(
    parameter_dict=parameter_dict, 
    simulation_options=simulation_options
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=result.index,
    y=result["Ti"],
    mode='lines',
    line_color='brown',
    name="Model_results"
))

fig.add_trace(go.Scatter(
    x=result.index,
    y=df["T_int"],
    fill='tonexty', 
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Simulation result',
    xaxis_title='Date',
    yaxis_title='Temperature [°C]')

fig.show()

## Sensitity analysis
We use first a sensitivity analysis to "rank" the parameters by order of influence on the error between measured temperature and model prediction.

The chosen error function is the CV_RMSE. The formula for CV_RMSE is given by:

$$
CV\_RMSE = \frac{RMSE}{\bar{y}}
$$

Where:
- *RMSE* is the root mean squared error,
- *bar{y}* is the mean of the observed values.

The RMSE is calculated as:

$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

Where:
- *n* is the number of observations,
- *y_i* is the observed value for the \( i \)-th observation,
- *hat{y}_i* is the predicted value for the \( i \)-th observation.

The CV_RMSE measures the variation of the RMSE relative to the mean of the observed values. It provides a standardized measure of the error, which can be useful for comparing the performance of different models across different datasets.


The chosen parameters are all the model parameters θ=(Ri,Ro,Ci,Ce,Ai,Ae). They must be described using a dictionary.

As you can see, simulation results are very inaccurate and give very high temperatures.  We should use the measurement to obtain optimal values of parameters. 

In [ ]:
from corrai.base.parameter import Parameter

In [ ]:
id_param = [
    {Parameter.NAME: "Ri", Parameter.INTERVAL: (0.004, 0.006), Parameter.TYPE: "Real"},
    {Parameter.NAME: "Ro", Parameter.INTERVAL: (0.045, 0.055), Parameter.TYPE: "Real"},
    {Parameter.NAME: "Ci", Parameter.INTERVAL: (4e6, 7e6), Parameter.TYPE: "Real"},
    {Parameter.NAME: "Ce", Parameter.INTERVAL: (1.3e7, 2.3e7), Parameter.TYPE: "Real"},
    {Parameter.NAME: "Ai", Parameter.INTERVAL: (-2, 0.5), Parameter.TYPE: "Real"},
    {Parameter.NAME: "Ae", Parameter.INTERVAL: (-2, 3), Parameter.TYPE: "Real"},
]

We can now use a <code>SAnalysis</code> to set-up the study. We have to pass
the <code>Simulator</code> previously describe, along with the corresponding
 problem description. A Sensitivity Analysis is also required. In this case we choose Sobol
, as there is few uncertain parameter.

*Note: for now only <code>SOBOL</code>, <code>FAST</code>, <code>RDB_FAST</code>, 
and <code>MORRIS</code> methods are implemented.*

In [ ]:
from corrai.sensitivity import SAnalysis, Method

sa_study = SAnalysis(
    parameters_list=id_param,
    method=Method.RDB_FAST,
)

We draw a sample of parameters to simulate. Each method has its sampling method.
Please see SALib documentation for further explanation (https://salib.readthedocs.io/en/latest/index.html)

Note: Convergence properties of the Sobol' sequence is only valid if
        `N` (100) is equal to `2^n`.
        N (int) – The number of samples to generate. Ideally a power of 2 and <= skip_values.

In [ ]:
sa_study.draw_sample(
    n=512, 
    sampling_kwargs={
#         "scramble":True,
#         "skip_values":2,
#         "calc_second_order": True, 
#         "seed": 42
    }
)

The sample is available as a 2d array <code>sa_study.sample</code>. Lines are simulations
to run and columns are parameters values.

Let's run the simulations. **CAREFUL depending on your computer, it can take a long time (up to 30')**

In [ ]:
sa_study.evaluate(
    model = simple_rc, 
    simulation_options=simulation_options,
    n_cpu=1
)

We can plot all simulations in one graph and compare the simulated internal temperature to measured T_int. Argument <code>show_legends</code> can be set to True if you want see associated parameters values.

In [ ]:
from corrai.sensitivity import plot_sample

In [ ]:
plot_sample(
    sample_results=sa_study.sample_results,
    ref=inputs["T_int"],
    indicator="Ti",
    show_legends=True,
)

We can also look at results using a parallel coordinate plots <code>plot_pcp</code> for all parameters values and an indicator. 
 
- A simple indicator would be the mean (by Default) of "Ti"
- Or you can choose a reference and an aggregation method. 

For a small cv_rmse, values of parameters are  quite spread out on their intervals. It can mean that optimization might not be easy, as multiple values provide the same minimization. 

In [ ]:
from corrai.sensitivity import plot_pcp
from corrai.metrics import nmbe, cv_rmse

plot_pcp(
    sample_results=sa_study.sample_results,
    parameters=id_param,
    indicators=['Ti'],
    reference=inputs["T_int"],
    reference_aggregation=nmbe,
)

Now that all simulations are run, we can analyze these results regarding an indicator with method <code>analyze</code>. We can either choose an aggregation method on Ti (for instance the average temperature throughout the timerange of simulations), or an aggregation function between predicted and measured temperatures.

In [ ]:
from corrai.metrics import cv_rmse

sa_study.analyze(
    indicator="Ti",
    reference_df=inputs["T_int"].to_frame(),
    agg_method=cv_rmse,
)

We can now have a look at the sensitivity analysis results.
They are stored in <code>sensitivity_results</code>. It holds the output formatted
by <code>SALib</code>.

According to the method used, we can sum the indices of partial or total order. You can do it manually or using method <code>calculate_sensitivity_summary</code>.

In [ ]:
sa_study.calculate_sensitivity_summary()

The sum of all the indices is very close to 1. Also, the mean confidence interval
seems to be very low. Results of the sensitivity analysis appear to be robust.
We can also plot the results. For Morris analysis, the elementary effect can be observed using <code>plot_morris_scatter</code> 

In [ ]:
from corrai.sensitivity import plot_morris_scatter 
plot_morris_scatter(salib_res=sa_study.sensitivity_results, title='Elementary effects', unit='J', autosize=True) 

For Sobol analysis, you can direcltly use  <code>plot_sobol_st_bar</code> 

In [ ]:
from corrai.sensitivity import plot_sobol_st_bar
plot_sobol_st_bar(sa_study.sample_results)

Otherwise:

In [ ]:
import plotly.graph_objects as go

# Retrieve results
S1 = sa_study.sensitivity_results['S1']
S1_conf = sa_study.sensitivity_results['S1_conf']
names = sa_study.sensitivity_results['names']

fig = go.Figure()

fig.add_trace(go.Bar(
    x=names,
    y=S1,
    error_y=dict(
        type='data',
        array=S1_conf,
        visible=True
    ),
    marker=dict(color='orange')  
))

fig.update_layout(
    title='Sensitivity indices with error bars',
    xaxis=dict(title='Parameter'),
    yaxis=dict(title='Indices')
)

# Afficher le graphique
fig.show()


# Conclusion on sensitivity analysis

The sensitivity analysis allows us to rank the influence of uncertain parameter
on an indicator. In this case we choose the $CV RMSE$ between model output
and measurement.

It shows here that the most influential parameters are Ai and Ae, which is comforting for their optimization as the parallel coordinate plot shows a rather limited range for small nmbe and cv_rmse for these parameters.

In the following chapter, we will see how to use corrai to identify the
optimal values for these parameters in order to fit the measurement.

# Optimization
Now we can find optimal values for these parameters, minimizing the cv_rmse between the measured indoor temperature and the output of the function we just defined. 
For this, we created in multi_optimize two classes, MyProblem and MyMixedProblem, to define problems for pymoo use. 
They take as arguments: 
- parameters (as they were defined with enumerators earlier in id_params). 
- list of object functions, as long as they have a "function" method (they can be useful for Modelica models transcripted into python, for instance)
- list of functions to minimize
- variable function names
- constraint names 

In Pymoo, each objective function is supposed to be minimized, and each constraint needs to be provided in the form of ≤0. For maximization, use inverse functions.

Let's define an aggregation function to minimize for our model, using as argument a parameter dictionary. 

In [ ]:
def RC_function(x_dict):
    model = SimpleRC()
    simulated_data = model.simulate(x_dict, simulation_options)
    y_pred = simulated_data["Ti"]
    y_true = simulation_options["dataframe"]["T_int"]
    return pd.Series(cv_rmse(y_pred, y_true), dtype="float64", index=["Ti"])

Now, instanciate the pymoo problem. For problems using Real parameters, use <code>MyProblem</code>. For problems with mixes variables (Parameter.TYPE can be either Real, Integer, Descrete, Choice), use <code>MyMixedProblem</code>.

In [ ]:
from corrai.multi_optimize import MyProblem 

problem = MyProblem(
                parameters=id_param,
                obj_func_list=[],
                func_list=[RC_function],
                function_names=["Ti"],
                constraint_names=[],
            )

In [ ]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems import get_problem
from pymoo.optimize import minimize

algorithm = GA(
    pop_size=100,
    eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

The estimated parameters seem consistent with our expectations. We can compare the profile of measured indoor temperature with the output that the model predicts given the identified optimal parameters. 

In [ ]:
parameter_dict = {param: res.X[i] for i, param in enumerate(["Ri", "Ro", "Ci", "Ce", "Ai", "Ae"])}

In [ ]:
result_optim = simple_rc.simulate(
    parameter_dict=parameter_dict, 
    simulation_options=simulation_options
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=result_optim.index,
    y=result_optim["Ti"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="Optim_results"
))

fig.add_trace(go.Scatter(
    x=result.index,
    y=df["T_int"],
    fill='tonexty', 
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Optimization vs. Reality ',
    xaxis_title='Date',
    yaxis_title='Temperature [°C]')

fig.show()

## Conclusion on the optimization
We have executed an algorithm and obtained a solution set.

As a remininder, you can use MyProblem for more than one objective, and with constraints as well. Choose wisely the suited algorithm (https://pymoo.org/algorithms/list.html#nb-algorithms-list) as well as their customization (poopulation size, generation, termination, etc.). 

Also, use convergence analysis by keeping track of the optimization progress, and storing information while running the algorithms using the argument <code>save_history

In [ ]:
res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False,
               save_history=True)

hist = res.history
print(len(hist))

In [ ]:
from pymoo.util.running_metric import RunningMetricAnimation

running = RunningMetricAnimation(delta_gen=5,
                        n_plots=3,
                        key_press=False,
                        do_show=True)

for algorithm in res.history[:15]:
    running.update(algorithm)